## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import numpy as np
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,57.20,41,20,19.57,few clouds
1,1,Nazare,BR,-13.0350,-39.0144,94.80,39,93,8.21,overcast clouds
2,2,Tasiilaq,GL,65.6145,-37.6368,32.00,93,75,44.87,snow
3,3,Carnarvon,AU,-24.8667,113.6333,80.60,100,40,11.50,scattered clouds
4,4,Rikitea,PF,-23.1203,-134.9692,77.47,75,100,13.15,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
3,3,Carnarvon,AU,-24.8667,113.6333,80.60,100,40,11.50,scattered clouds
4,4,Rikitea,PF,-23.1203,-134.9692,77.47,75,100,13.15,overcast clouds
6,6,Vaini,TO,-21.2000,-175.2000,73.40,100,90,11.50,overcast clouds
13,13,Kavaratti,IN,10.5669,72.6420,81.21,66,0,10.00,clear sky
16,16,Georgetown,MY,5.4112,100.3354,84.20,74,20,5.75,few clouds
21,21,Buritizeiro,BR,-17.3511,-44.9622,78.93,82,95,8.30,overcast clouds
22,22,Avarua,CK,-21.2078,-159.7750,75.20,94,20,2.30,few clouds
23,23,Atuona,PF,-9.8000,-139.0333,80.17,77,57,22.41,broken clouds
26,26,Hermanus,ZA,-34.4187,19.2345,73.00,79,20,4.00,few clouds
30,30,Cape Town,ZA,-33.9258,18.4232,75.00,68,20,19.57,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        4
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_dropna = preferred_cities_df.dropna()
preferred_cities_dropna.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_dropna[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Carnarvon,AU,80.60,scattered clouds,-24.8667,113.6333,
4,Rikitea,PF,77.47,overcast clouds,-23.1203,-134.9692,
6,Vaini,TO,73.40,overcast clouds,-21.2000,-175.2000,
13,Kavaratti,IN,81.21,clear sky,10.5669,72.6420,
16,Georgetown,MY,84.20,few clouds,5.4112,100.3354,
21,Buritizeiro,BR,78.93,overcast clouds,-17.3511,-44.9622,
22,Avarua,CK,75.20,few clouds,-21.2078,-159.7750,
23,Atuona,PF,80.17,broken clouds,-9.8000,-139.0333,
26,Hermanus,ZA,73.00,few clouds,-34.4187,19.2345,
30,Cape Town,ZA,75.00,few clouds,-33.9258,18.4232,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        hotel_df.loc[index, "Hotel Name"] = np.nan
        

In [9]:
#hotels #Checking json readout

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 5.6599906, 'lng': -53.7770331},
    'viewport': {'northeast': {'lat': 5.661511230291501,
      'lng': -53.7757037197085},
     'southwest': {'lat': 5.658813269708497, 'lng': -53.7784016802915}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'LE SAMANA',
   'photos': [{'height': 1536,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108780863650213203327">LE SAMANA</a>'],
     'photo_reference': 'ATtYBwK2MrQqZuELzfY7zC3ZyHb6Sctx17KRdx83wo7eLwdWs2sRcK5l2p_Mnkfh6whHThtXzd4rxefMGRR1d55g4eNPhNbcQmGub-GGaLx2xnlxdToIoGYDMdczHp2W3wpiS42SmxQm4lNQ_iY7Q90oiLbBz-MCSAWvDK7fYGZ28VeJHa0J',
     'width': 2048}],
   'place_id': 'ChIJT6mMaNTdDo0RI7XHozHbU1Y',
   'plus_code': {'compound_code': 'M65F+X5 Mana, French Guiana',
    'global_code': '68Q8M65F+X5'},
   'rating': 4,
   'reference': 'ChIJT6mMaNTdDo0RI7XHozHbU1Y

In [10]:
# Checking for null values
#hotel_df.isnull().sum() #21 without

City            0
Country         0
Max Temp        0
Description     0
Lat             0
Lng             0
Hotel Name     21
dtype: int64

In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Carnarvon,AU,80.60,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon
4,Rikitea,PF,77.47,overcast clouds,-23.1203,-134.9692,Pension Maro'i
6,Vaini,TO,73.40,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort
13,Kavaratti,IN,81.21,clear sky,10.5669,72.6420,Hotels in Lakshadweep Islands
16,Georgetown,MY,84.20,few clouds,5.4112,100.3354,Cititel Penang


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig #figure would not display

Figure(layout=FigureLayout(height='420px'))